In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import sys

In [3]:
def pytorch_init():
    device_id = 1
    torch.cuda.set_device(device_id)
    
    # Sanity checks
    assert torch.cuda.current_device() == 1, 'Using wrong GPU'
    assert torch.cuda.device_count() == 2, 'Cannot find both GPUs'
    assert torch.cuda.get_device_name(0) == 'GeForce RTX 2080 Ti', 'Wrong GPU name'
    assert torch.cuda.is_available() == True, 'GPU not available'
    return torch.device('cuda', device_id)
    
device = pytorch_init()

# Main building blocks of PyTorch
- Module
- Sequential
- ModuleList

# Module 
The Module is the main building block, it defines the base class for all neural networks and you **MUST** subclass it. Below we make a simple CNN classifier with an encoding part that uses two layers with $3 \times 3$ convs + barchnorm + ReLU.

In [5]:
class MyCNNClassifier(nn.Module):
    def __init__(self, in_c: int, n_classes: int):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=in_c, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        
        self.fc1 = nn.Linear(in_features=32*28*28, out_features=1024)
        self.fc2 = nn.Linear(in_features=1024, out_features=n_classes)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        
        x = x.view(x.size(0), -1) # Flatten, .size(0) is same as .shape[0]
        
        x = self.fc1(x)
        x = F.sigmoid(x)
        x = self.fc2(x)
        return x

In [6]:
model = MyCNNClassifier(1, 10)
print(model)

MyCNNClassifier(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=25088, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=10, bias=True)
)


### Problem is that it is not that scalable
If we want to add a layer we have to declare and initialize another layer, then code what to do with it in the forward method. Also if we have some common block that we want to use in another model, we have to write it all over again.

# Sequential
Sequential is a container of Moules that can be stacked togeter and run at the same it. It very similar to Keras' sequential API. It can often yield cleaner code if used correctly

In [7]:
class MyCNNClassifier(nn.Module):
    def __init__(self, in_c: int, n_classes: int):
        super().__init__()
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(in_c, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLu()
        )
        
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLu()
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(32 * 28 * 28, 1024),
            nn.Sigmoid(),
            nn.Linear(1024, n_classed)
        )
        
    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)

        x = x.view(x.size(0), -1) # flat
        
        x = self.decoder(x)
        
        return x